## Best Predictor in Predicting Tips: A STAT 301 Case Study Project

### Introduction

##### Data Description
This dataset is obtained from https://www.kaggle.com/datasets/saurabhbadole/restaurant-tips-dataset.  
Collected from a popular restaurant over a specified period, this dataset looks into the tipping behavior in a restaurant setting. In this dataset, there are in total 244 observations, and 7 variables.    
Table of variables used:
| Variable Name | Discription | Type |
| :---: | :---: | :---: |
|total_bill| total bill amount in dollars|Numerical, independent variable|
|sex| Gender of the person paying the bill (Male/Female)|Categorical, independent variable|
|smoker|Whether the person is a smoker (Yes/No) |Categorical, independent variable|
|day|Day of the week when the transaction took place (Thur/Fri/Sat/Sun)|Categorical, independent variable|
|time|Time of day when the transaction took place (Lunch/Dinner)|Categorical, independent variable|
|size|Size of the dining party (number of people)|Numerical, independent variable|
|tip|Tip amount in dollars|Numerical, target variable|

##### Research Question

*What are the best predictors in the dataset that best*

### Methods and Results

#### Exploratory Data Analysis (EDA)

In [1]:

url <- "https://raw.githubusercontent.com/JustinTrenchcoat/STAT_301_Personal_Project/refs/heads/main/tips.csv"

data_tips <- read.csv(url(url), header = TRUE)
head(data_tips)
ncol <- ncol(data_tips)
norw <- nrow(data_tips)

,total_bill,tip,sex,smoker,day,time,size
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4
6,25.29,4.71,Male,No,Sun,Dinner,4


#### Methods: Plan

### Discussion

### References

Badole, Saurabh. (2025, March). *Restaurant Tips Dataset*. Retrieved from [https://www.kaggle.com/datasets/saurabhbadole/restaurant-tips-dataset](https://www.kaggle.com/datasets/saurabhbadole/restaurant-tips-dataset)